# Regularization: LASSO & RIDGE

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')


In [2]:
train_df = pd.read_csv('./datasets/lasso_ridge_train.csv')
test_df = pd.read_csv('./datasets/lasso_ridge_test.csv')

train_df.set_index('id', inplace = True)
test_df.set_index('id', inplace = True)

In [3]:
train_df.head()

,ms_subclass,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,...,Timber,Veenker,A (agr),C (all),FV,I (all),RH,RL,screen_porch*wood_deck_sf,fireplaces*totrms_abvgrd
id,,,,,,,,,,,,,,,,,,,,,
544,60,43.0,11492,Pave,NoAccess,IR1,Lvl,AllPub,CulDSac,Gtl,...,0,0,0,0,0,0,0,1,0,8
153,20,68.0,7922,Pave,NoAccess,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,1,0,0
318,60,73.0,9802,Pave,NoAccess,Reg,Lvl,AllPub,Inside,Gtl,...,1,0,0,0,0,0,0,1,0,0
255,50,82.0,14235,Pave,NoAccess,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,1,0,0
138,20,137.0,16492,Pave,NoAccess,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,0,0,1,0,6


In [4]:
test_df.head()

,pid,ms_subclass,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,...,Timber,Veenker,A (agr),C (all),FV,I (all),RH,RL,screen_porch*wood_deck_sf,fireplaces*totrms_abvgrd
id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,69.000000,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,0,0,0
2718,905108090,90,69.545961,9662,Pave,NoAccess,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,0,0
2414,528218130,60,58.000000,17104,Pave,NoAccess,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,0,7
1989,902207150,30,60.000000,8520,Pave,NoAccess,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,0,0,0
625,535105100,20,69.545961,9500,Pave,NoAccess,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,0,12


In [5]:
#Fix up what I will put to my features here:

features = train_df.select_dtypes(exclude='object').drop(columns=['saleprice', 'bsmtfin_sf_2', 'central_air', 'low_qual_fin_sf', 'mo_sold', '3ssn_porch', 
                'enclosed_porch', 'open_porch_sf', 'half_bath', 'full_bath', 'bsmt_half_bath' ]).columns.to_list()
len(features)

60

In [6]:
#Define X and y
X = train_df[features]      #our training data 
y = train_df['saleprice']   #what we want to predict

In [7]:
X.columns

Index(['ms_subclass', 'lot_frontage', 'lot_area', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'mas_vnr_area',
       'bsmtfin_sf_1', 'bsmt_unf_sf', 'total_bsmt_sf', '1st_flr_sf',
       '2nd_flr_sf', 'gr_liv_area', 'bsmt_full_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'totrms_abvgrd', 'fireplaces', 'garage_yr_blt',
       'garage_cars', 'garage_area', 'wood_deck_sf', 'screen_porch',
       'pool_area', 'misc_val', 'yr_sold', 'Blueste', 'BrDale', 'BrkSide',
       'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'Greens',
       'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NPkVill', 'NWAmes', 'NoRidge',
       'NridgHt', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst',
       'StoneBr', 'Timber', 'Veenker', 'A (agr)', 'C (all)', 'FV', 'I (all)',
       'RH', 'RL', 'screen_porch*wood_deck_sf', 'fireplaces*totrms_abvgrd'],
      dtype='object')

In [8]:
#Split our data into training and testing sets-----------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

In [10]:
# Scale our data!
sc = StandardScaler()
Z_train = sc.fit_transform(X_train)
Z_test = sc.transform(X_test)

In [11]:
print(f'Z_train shape is: {Z_train.shape}')
print(f'y_train shape is: {y_train.shape}')
print(f'Z_test shape is: {Z_test.shape}')
print(f'y_test shape is: {y_test.shape}')

Z_train shape is: (1257, 60)
y_train shape is: (1257,)
Z_test shape is: (420, 60)
y_test shape is: (420,)


In [12]:
# OLS so that we can compare our score to RIDGE and LASSO!!------------------------------------------

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lr = LinearRegression()
lr.fit(Z_train, y_train)

LinearRegression()

In [15]:
# How does the model score on the training and test data?
print(lr.score(Z_train, y_train))
print(lr.score(Z_test, y_test))

0.9175616782333031
0.9021036853516992


In [16]:
#   lets see our baseline model RMSE:
from sklearn.metrics import mean_squared_error

In [17]:
baseline_pred = [y_train.mean()]*len(y_test)
mean_squared_error(y_test, baseline_pred)** 0.5 #just like the prev... our baseline prediction is off by 75k

78614.86707985436

In [18]:
train_pred = lr.predict(Z_train)
test_pred = lr.predict(Z_test)

#Eval lr model using RMSE
print(mean_squared_error(y_train,train_pred)**0.5  ) # 
print(lr.score(Z_test, y_test))

21374.726859588136
0.9021036853516992


## Ridge

In [19]:
#ridge reduces our coef size!

In [20]:
from sklearn.linear_model import Ridge

In [21]:
#Instantiate
ridge = Ridge(alpha= 600)  #Linear Reg model nad L2 regression (refer to the chart in the lesson for Regularization)

#Fit
ridge.fit(Z_train, y_train)

#Evaluate model using R2
print(f"Ridge Training Score: {ridge.score(Z_train,y_train)}")
print(f"Ridge Testing Score: {ridge.score(Z_test, y_test)}")


# Generate predictions on the test set and evaluate using MSE
pred_ridge = ridge.predict(Z_test)
mean_squared_error(y_test, pred_ridge)**0.5  ##RMSE  #hmm this isnt a good score

Ridge Training Score: 0.9004227154373543
Ridge Testing Score: 0.8877772319998067


26317.819318727496

In [22]:
# Looks like our model does good on both our training and test set!

In [23]:
ridge.coef_

array([-2240.23443499,  2751.15350555,  4082.89976707, 11555.54706138,
        2563.96257175,  6969.14582608,  6107.9116097 ,  4385.57505943,
        6061.18632859,   532.69909923,  6721.2104134 ,  7091.49798596,
        4650.96890239,  9409.79400086,  2937.47704449, -1601.00743538,
       -2313.89515235,  4006.55611482,  1998.00441196, -1445.84800442,
        4500.12635683,  5351.9739518 ,  2720.71785212,  1150.12941073,
          40.13198597,   420.23876209,  -532.0562452 ,  -163.6336746 ,
        -739.00279073,   352.31740576,  -406.82896946,   233.44861895,
        2748.21101487, -1896.50470647,  -185.20443724,   -45.45721495,
        -692.39064102,  -113.44715691, -1474.58606094, -3231.26072594,
        -250.77253968, -2790.3378099 ,  2210.71282301,  6289.58789931,
       -1740.3179374 ,  -410.24252219, -1925.83866633, -1107.1983302 ,
        2347.935556  ,  5316.33657357,  1180.7595657 ,  1607.20729518,
        -912.87613334,  -603.4415058 ,  1676.6833956 ,  -134.97581497,
      

### Brute force the answer to find our optimal alphas:

In [24]:
from sklearn.linear_model import RidgeCV

In [25]:
np.linspace(0,10, 21)  #returns an array of n-values evenly spaced b/w start and stop

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. ])

In [26]:
np.logspace(0,3, 20)

array([   1.        ,    1.43844989,    2.06913808,    2.97635144,
          4.2813324 ,    6.15848211,    8.8586679 ,   12.74274986,
         18.32980711,   26.36650899,   37.92690191,   54.55594781,
         78.47599704,  112.88378917,  162.37767392,  233.57214691,
        335.98182863,  483.29302386,  695.19279618, 1000.        ])

In [27]:
#set up a list of ridge alphas to check (can use either np.linspace or np.logspace)
ridge_alphas = np.logspace(0, 3, 100)


#cross-validate over our list of ridge alphas
ridge_cv = RidgeCV(alphas = ridge_alphas, scoring='r2', cv=5)

#fit model using best ridge alpha
ridge_cv.fit(Z_train, y_train)

RidgeCV(alphas=array([   1.        ,    1.07226722,    1.149757  ,    1.23284674,
          1.32194115,    1.41747416,    1.51991108,    1.62975083,
          1.7475284 ,    1.87381742,    2.009233  ,    2.15443469,
          2.3101297 ,    2.47707636,    2.65608778,    2.84803587,
          3.05385551,    3.27454916,    3.51119173,    3.76493581,
          4.03701726,    4.32876128,    4.64158883,    4.97702356,
          5.33669923,    5.72236766,    6.13590727,    6.57933225,
          7.05480231,    7.56463328,    8.11130831,    8...
        151.9911083 ,  162.97508346,  174.75284   ,  187.38174229,
        200.92330026,  215.443469  ,  231.01297001,  247.7076356 ,
        265.60877829,  284.80358684,  305.38555088,  327.45491629,
        351.11917342,  376.49358068,  403.70172586,  432.87612811,
        464.15888336,  497.70235643,  533.66992312,  572.23676594,
        613.59072734,  657.93322466,  705.48023107,  756.46332755,
        811.13083079,  869.74900262,  932.60334688, 10

In [28]:
# Here are the optimal values of alpha!
ridge_cv.alpha_

37.64935806792467

In [29]:
#we are introducing some bias.. we are making our model slighltly worse on our training set to reduce our variuance by introducing the bias we change up our training score.. 
#in exhacnge we reduce our variance by making our test scores better b

## Evaluate our model on both training & testing data
print(ridge_cv.score(Z_train, y_train))   #whenver you return a .score in a regression model its using R2
print(ridge_cv.score(Z_test, y_test))

0.9168579102571669
0.9019913361659186


In [30]:
# we see our ridge_cv scores does slighlty better then our ridge score

In [31]:
## Baseline scores
## Evaluate OLS model using RMSE
print("---Baseline scores---")
print(mean_squared_error(y_train, lr.predict(Z_train))**0.5)
print(mean_squared_error(y_test, lr.predict(Z_test))**0.5)


#
print('------------')
#recall RMSE you want your score to be as near to 0.. 


## Evaluate Ridge model using RMSE
print(mean_squared_error(y_train, ridge_cv.predict(Z_train))**0.5)
print(mean_squared_error(y_test, ridge_cv.predict(Z_test))**0.5)

---Baseline scores---
21374.726859588136
24580.60592217033
------------
21465.769953484476
24594.70665310988


In [32]:
#seeing comparing :

print(" OLS ".center(18, "="))
print(lr.score(Z_train, y_train))
print(lr.score(Z_test, y_test))
print('Dif b/w scores: ',lr.score(Z_train, y_train) - lr.score(Z_test, y_test))
print()
print(" Ridge ".center(18, "="))
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))
print('Dif b/w scores :',ridge_cv.score(Z_train, y_train) - ridge_cv.score(Z_test, y_test))

====== OLS =======
0.9175616782333031
0.9021036853516992
Dif b/w scores:  0.015457992881603988

===== Ridge ======
0.9168579102571669
0.9019913361659186
Dif b/w scores : 0.014866574091248252


**RIDGE comes out slightly ahead, with less variance**

In [33]:
lr.coef_

array([-6.40822350e+03,  2.05545022e+03,  5.71175148e+03,  1.56731812e+04,
        5.98321613e+03,  1.51365868e+04,  3.02461768e+03,  3.27246839e+03,
        7.15675388e+03, -8.40360025e+02,  6.32021132e+03,  1.74494220e+04,
        1.77901170e+04,  4.68832787e+03,  2.58620310e+03, -4.78637891e+03,
       -1.00152561e+03,  4.75272726e+02, -6.87480621e+03, -4.54161234e+03,
        3.20343274e+03,  5.49291861e+03,  1.80276614e+03,  1.09836715e+03,
        1.04916702e+02,  3.10892630e+02, -8.60806401e+02,  7.11844313e+02,
        7.59617254e+02,  2.78481313e+03, -6.06932815e+02, -9.62371947e+02,
        5.17475967e+03, -9.16496023e+02, -4.01445901e+02,  2.60741007e+00,
        2.20870046e+03,  1.24963107e+03, -1.70613465e+03, -2.49160057e+03,
        1.02486643e+03, -4.22489429e+03,  1.40766388e+03,  7.79999454e+03,
        1.76164563e+03,  1.07048792e+03, -1.32441196e+03, -2.15047782e+03,
        2.61543066e+03,  6.50076012e+03,  1.06215806e+03,  1.84575509e+03,
       -1.27605229e+03, -

In [34]:
ridge_cv.coef_  #we shrinked down our coef a bit through ridge

array([-5.87894369e+03,  2.23461146e+03,  5.55126984e+03,  1.51794124e+04,
        5.48145183e+03,  1.30162600e+04,  3.72389118e+03,  3.57835878e+03,
        7.13695190e+03, -7.55382205e+02,  6.56361028e+03,  1.00082097e+04,
        9.52893057e+03,  1.24294047e+04,  2.58861448e+03, -4.34536841e+03,
       -1.47104111e+03,  1.75542944e+03, -2.05991015e+03, -4.13933117e+03,
        3.34508578e+03,  5.58297329e+03,  1.91640354e+03,  1.16014751e+03,
        1.78741311e+02,  4.12310356e+02, -8.30284294e+02,  2.90620649e+02,
        1.07357635e+02,  1.57477344e+03, -8.63541407e+02, -1.24120310e+03,
        4.29620205e+03, -1.82034392e+03, -6.85456262e+02, -1.22217581e+02,
        6.86661522e+02,  6.89805237e+02, -2.15539155e+03, -3.53773061e+03,
        6.04449230e+02, -4.44542391e+03,  1.46551406e+03,  7.37022278e+03,
       -2.64841365e+02,  2.85899124e+02, -1.97594688e+03, -2.37904613e+03,
        2.32154384e+03,  6.32121217e+03,  8.54420390e+02,  1.71257749e+03,
       -1.16997216e+03, -

## LASSO

In [35]:
from sklearn.linear_model import Lasso, LassoCV

In [36]:
## We can also use np.logspace to generate values less than 1
np.logspace(-3, 0, 100)  ## that negative value means=  10^-3 all the way to 10^0

array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.00811131, 0.00869749, 0.00932603, 0.01      , 0.01072267,
       0.01149757, 0.01232847, 0.01321941, 0.01417474, 0.01519911,
       0.01629751, 0.01747528, 0.01873817, 0.02009233, 0.02154435,
       0.0231013 , 0.02477076, 0.02656088, 0.02848036, 0.03053856,
       0.03274549, 0.03511192, 0.03764936, 0.04037017, 0.04328761,
       0.04641589, 0.04977024, 0.05336699, 0.05722368, 0.06135907,
       0.06579332, 0.07054802, 0.07564633, 0.08111308, 0.0869749 ,
       0.09326033, 0.1       , 0.10722672, 0.1149757 , 0.12328467,
       0.13219411, 0.14174742, 0.15199111, 0.16297508, 0.17475

In [37]:
# Set up a list of Lasso alphas to check.
l_alphas = np.logspace(-3,0,100)

# Cross-validate over our list of Lasso alphas
## since we're using small alpha values, we might want to increase the number of iterations and/or the tolerence
## we can try max_iter = 5000, and tol = 0.00015
lasso_cv = LassoCV(alphas = l_alphas, cv = 5, max_iter=5000, tol = 0.00015) #cv is cross validation( the slices)
          #to not get the warnings .. incr the parameters as noahs comment said ^ for max_iter and tol.. these 2arg says to run the ftn longer more iterations

# Fit model using the best lasso alpha!
lasso_cv.fit(Z_train, y_train)


LassoCV(alphas=array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.008...
       0.13219411, 0.14174742, 0.15199111, 0.16297508, 0.17475284,
       0.18738174, 0.2009233 , 0.21544347, 0.23101297, 0.24770764,
       0.26560878, 0.28480359, 0.30538555, 0.32745492, 0.35111917,
       0.37649358, 0.40370173, 0.43287613, 0.46415888, 0.49770236,
       0.53366992, 0.57223677, 0.61359073, 0.65793322, 0.70548023,
       0.75646333, 0.81113083, 0.869749  , 0.93260335, 1.        ]),
        cv=5, max_iter=5000, tol=0.00015)

In [38]:
lasso_cv.alpha_ #optimal value of alpha!

1.0

In [39]:
lasso_cv.alphas_

array([1.        , 0.93260335, 0.869749  , 0.81113083, 0.75646333,
       0.70548023, 0.65793322, 0.61359073, 0.57223677, 0.53366992,
       0.49770236, 0.46415888, 0.43287613, 0.40370173, 0.37649358,
       0.35111917, 0.32745492, 0.30538555, 0.28480359, 0.26560878,
       0.24770764, 0.23101297, 0.21544347, 0.2009233 , 0.18738174,
       0.17475284, 0.16297508, 0.15199111, 0.14174742, 0.13219411,
       0.12328467, 0.1149757 , 0.10722672, 0.1       , 0.09326033,
       0.0869749 , 0.08111308, 0.07564633, 0.07054802, 0.06579332,
       0.06135907, 0.05722368, 0.05336699, 0.04977024, 0.04641589,
       0.04328761, 0.04037017, 0.03764936, 0.03511192, 0.03274549,
       0.03053856, 0.02848036, 0.02656088, 0.02477076, 0.0231013 ,
       0.02154435, 0.02009233, 0.01873817, 0.01747528, 0.01629751,
       0.01519911, 0.01417474, 0.01321941, 0.01232847, 0.01149757,
       0.01072267, 0.01      , 0.00932603, 0.00869749, 0.00811131,
       0.00756463, 0.0070548 , 0.00657933, 0.00613591, 0.00572

In [40]:
print(" OLS ".center(18, "="))
print(lr.score(Z_train, y_train))  #you can explain 90% 
print(lr.score(Z_test, y_test))
print('Dif: ', lr.score(Z_train, y_train) - lr.score(Z_test, y_test))  #
print()
print(" Ridge ".center(18, "="))
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))
print('Dif: ', ridge_cv.score(Z_train, y_train) - ridge_cv.score(Z_test, y_test))
print()
print(" Lasso ".center(18, "="))
print(lasso_cv.score(Z_train, y_train))
print(lasso_cv.score(Z_test, y_test))
print('Dif: ', lasso_cv.score(Z_train, y_train) - lasso_cv.score(Z_test, y_test))

====== OLS =======
0.9175616782333031
0.9021036853516992
Dif:  0.015457992881603988

===== Ridge ======
0.9168579102571669
0.9019913361659186
Dif:  0.014866574091248252

===== Lasso ======
0.9175616197708721
0.902122421276135
Dif:  0.015439198494737183


**Ridge is doing slighlty better then the other models.**

In [41]:
##**R-squared interpretation: The proportion of variability in your target that is explained by the features in your model, relative to a model with no predictors**

R2 score: 90% of variability in my target variable is explained in by the features in my model relative to a model with no predictors at all.

In [42]:
## Evaluate OLS model using RMSE
print(" OLS - RMSE ".center(18, "="))
print('Trained: ',mean_squared_error(y_train, lr.predict(Z_train))**0.5)
print('Tested: ', mean_squared_error(y_test, lr.predict(Z_test))**0.5)
print('Dif: ', mean_squared_error(y_train, lr.predict(Z_train))**0.5 - mean_squared_error(y_test, lr.predict(Z_test))**0.5 )
print()
## Evaluate Ridge model using RMSE
print(" Ridge - RMSE ".center(18, "="))
print('Trained: ',mean_squared_error(y_train, ridge_cv.predict(Z_train))**0.5)
print('Tested: ', mean_squared_error(y_test, ridge_cv.predict(Z_test))**0.5)
print('Dif: ', mean_squared_error(y_train, ridge_cv.predict(Z_train))**0.5 - mean_squared_error(y_test, ridge_cv.predict(Z_test))**0.5 )
print()
## Evaluate LASSO model using RMSE
print(" LASSO - RMSE ".center(18, "="))
print('Trained: ', mean_squared_error(y_train, lasso_cv.predict(Z_train))**0.5)
print('Tested: ', mean_squared_error(y_test, lasso_cv.predict(Z_test))**0.5)
print('Dif: ', mean_squared_error(y_train, lasso_cv.predict(Z_train))**0.5 - mean_squared_error(y_test, lasso_cv.predict(Z_test))**0.5 )

=== OLS - RMSE ===
Trained:  21374.726859588136
Tested:  24580.60592217033
Dif:  -3205.879062582193

== Ridge - RMSE ==
Trained:  21465.769953484476
Tested:  24594.70665310988
Dif:  -3128.936699625403

== LASSO - RMSE ==
Trained:  21374.73443869847
Tested:  24578.253625180998
Dif:  -3203.5191864825283


## Ridge vs Lasso

In [43]:
##look at the coef

In [44]:
## Check Ridge coefficients
ridge_cv.coef_   #ridge has the effect of reducing your coef size

array([-5.87894369e+03,  2.23461146e+03,  5.55126984e+03,  1.51794124e+04,
        5.48145183e+03,  1.30162600e+04,  3.72389118e+03,  3.57835878e+03,
        7.13695190e+03, -7.55382205e+02,  6.56361028e+03,  1.00082097e+04,
        9.52893057e+03,  1.24294047e+04,  2.58861448e+03, -4.34536841e+03,
       -1.47104111e+03,  1.75542944e+03, -2.05991015e+03, -4.13933117e+03,
        3.34508578e+03,  5.58297329e+03,  1.91640354e+03,  1.16014751e+03,
        1.78741311e+02,  4.12310356e+02, -8.30284294e+02,  2.90620649e+02,
        1.07357635e+02,  1.57477344e+03, -8.63541407e+02, -1.24120310e+03,
        4.29620205e+03, -1.82034392e+03, -6.85456262e+02, -1.22217581e+02,
        6.86661522e+02,  6.89805237e+02, -2.15539155e+03, -3.53773061e+03,
        6.04449230e+02, -4.44542391e+03,  1.46551406e+03,  7.37022278e+03,
       -2.64841365e+02,  2.85899124e+02, -1.97594688e+03, -2.37904613e+03,
        2.32154384e+03,  6.32121217e+03,  8.54420390e+02,  1.71257749e+03,
       -1.16997216e+03, -

In [45]:
## Check LASSO coeffiecients
lasso_cv.coef_

array([-6.40774013e+03,  2.05442079e+03,  5.71149505e+03,  1.56739322e+04,
        5.98210994e+03,  1.51226086e+04,  3.02492045e+03,  3.27384159e+03,
        7.16478730e+03, -8.31687763e+02,  6.31340048e+03,  1.73912851e+04,
        1.77283302e+04,  4.75709340e+03,  2.58596048e+03, -4.78139630e+03,
       -1.00213708e+03,  4.76552349e+02, -6.83484588e+03, -4.53917050e+03,
        3.19659688e+03,  5.49875272e+03,  1.80251360e+03,  1.09785885e+03,
        1.04830038e+02,  3.10436177e+02, -8.59754471e+02,  7.02226477e+02,
        7.45285189e+02,  2.76105791e+03, -6.11474440e+02, -9.76240785e+02,
        5.15919000e+03, -9.34926422e+02, -4.10818049e+02, -0.00000000e+00,
        2.17866421e+03,  1.23630250e+03, -1.71762976e+03, -2.51513744e+03,
        1.01688145e+03, -4.23359128e+03,  1.40121257e+03,  7.78819153e+03,
        1.72360164e+03,  1.05684161e+03, -1.33721431e+03, -2.15994288e+03,
        2.60329843e+03,  6.49438441e+03,  1.05458473e+03,  1.84167783e+03,
       -1.27382317e+03, -